## Advice from first week

Competition data is rather challenging, so the sooner you get yourself familiar with it - the better. You can start with submitting sample_submission.csv from "Data" page on Kaggle and try submitting different constants.

## Advice from second week

A good exercise is to reproduce previous_value_benchmark. As the name suggest - in this benchmark for the each shop/item pair our predictions are just monthly sales from the previous month, i.e. October 2015.

The most important step at reproducing this score is correctly aggregating daily data and constructing monthly sales data frame. You need to get lagged values, fill NaNs with zeros and clip the values into [0,20] range. If you do it correctly,  you'll get precisely 1.16777 on the public leaderboard.

Generating features like this is a necessary basis for more complex models. Also, if you decide to fit some model, don't forget to clip the target into [0,20] range, it makes a big difference.

## Advice from third week

You can get a rather good score after creating some lag-based features like in advice from previous week and feeding them into gradient boosted trees model.

Apart from item/shop pair lags you can try adding lagged values of total shop or total item sales (which are essentially mean-encodings). All of that is going to add some new information.

## Descriptions

You are provided with daily historical sales data. The task is to forecast the total amount of products sold in every shop for the test set. Note that the list of shops and products slightly changes every month. Creating a robust model that can handle such situations is part of the challenge.

## File descriptions

- sales_train.csv - the training set. Daily historical data from January 2013 to October 2015.
- test.csv - the test set. You need to forecast the sales for these shops and products for November 2015.
- sample_submission.csv - a sample submission file in the correct format.
- items.csv - supplemental information about the items/products.
- item_categories.csv  - supplemental information about the items categories.
- shops.csv- supplemental information about the shops.

## Data fields

- ID - an Id that represents a (Shop, Item) tuple within the test set
- shop_id - unique identifier of a shop
- item_id - unique identifier of a product
- item_category_id - unique identifier of item category
- item_cnt_day - number of products sold. You are predicting a monthly amount of this measure
- item_price - current price of an item
- date - date in format dd/mm/yyyy
- date_block_num - a consecutive month number, used for convenience. January 2013 is 0, February 2013 is 1,..., - October 2015 is 33
- item_name - name of item
- shop_name - name of shop
- item_category_name - name of item category


In [1]:
import pandas as pd
import re

pd.plotting.register_matplotlib_converters()
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import numpy as np

## 1. Preprocessing data

In [2]:
item_categories = pd.read_csv('readonly/final_project_data/item_categories.csv')
items = pd.read_csv('readonly/final_project_data/items.csv')
shops = pd.read_csv('readonly/final_project_data/shops.csv')

sales_train = pd.read_csv('readonly/final_project_data/sales_train.csv.gz')
test = pd.read_csv('readonly/final_project_data/test.csv.gz')

sample_submission = pd.read_csv('readonly/final_project_data/sample_submission.csv.gz')

In [3]:
sales_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2935849 entries, 0 to 2935848
Data columns (total 6 columns):
 #   Column          Dtype  
---  ------          -----  
 0   date            object 
 1   date_block_num  int64  
 2   shop_id         int64  
 3   item_id         int64  
 4   item_price      float64
 5   item_cnt_day    float64
dtypes: float64(2), int64(3), object(1)
memory usage: 134.4+ MB


In [4]:
items['item_category'] = items['item_category_id'].map(item_categories.set_index('item_category_id').to_dict()['item_category_name'])

In [5]:
sales_train['date'] = pd.to_datetime(sales_train['date'], format="%d.%m.%Y")
sales_train = sales_train.sort_values(by='date')
sales_train = sales_train.reset_index(drop=True)
sales_train = sales_train.drop_duplicates()
sales_train['shop_location'] = sales_train['shop_id'].map(shops.set_index('shop_id').to_dict()['shop_name'])
sales_train['item_category'] = sales_train['item_id'].map(items['item_category'].to_dict())
sales_train['item_name'] = sales_train['item_id'].map(items['item_name'].to_dict())
sales_train['day'] = sales_train['date'].dt.day
sales_train['month'] = sales_train['date'].dt.month
sales_train['dayofweek'] = sales_train['date'].dt.day_name()
sales_train['year'] = sales_train['date'].dt.year
sales_train['date_month'] = sales_train['date'].dt.to_period('M')
sales_train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,shop_location,item_category,item_name,day,month,dayofweek,year,date_month
0,2013-01-01,0,18,5823,2500.0,1.0,"Красноярск ТЦ ""Июнь""",Карты оплаты - PSN,Playstation Store пополнение бумажника: Карта ...,1,1,Tuesday,2013,2013-01
1,2013-01-01,0,27,5573,849.0,1.0,"Москва ТЦ ""МЕГА Белая Дача II""",Аксессуары - PS3,PS Move Motion Controller Charger (Зарядная ст...,1,1,Tuesday,2013,2013-01
2,2013-01-01,0,7,1006,399.0,1.0,"Воронеж ТРЦ ""Максимир""",Подарки - Развитие,3D Crystal Puzzle Дельфин XL,1,1,Tuesday,2013,2013-01
3,2013-01-01,0,19,17707,899.0,1.0,"Курск ТЦ ""Пушкинский""",Игры - PS3,Праздник спорта (Essentials) (только для PS Mo...,1,1,Tuesday,2013,2013-01
4,2013-01-01,0,14,19548,149.0,1.0,"Казань ТЦ ""ПаркХаус"" II",Кино - DVD,ТАКИЕ РАЗНЫЕ БЛИЗНЕЦЫ (регион),1,1,Tuesday,2013,2013-01


In [6]:
def get_category(row):
    if len(row.split(' - ')) == 1:
        return row
    
    category, subcategory = row.split(' - ')
    return category

def get_subcategory(row):
    if len(row.split(' - ')) == 1:
        return 'Прочие'
    
    category, subcategory = row.split(' - ')
    return subcategory

sales_train['category'] = sales_train['item_category'].apply(get_category)
sales_train['subcategory']  = sales_train['item_category'].apply(get_subcategory)

sales_train['location'] = sales_train['shop_location'].apply(lambda x: x.split()[0])
sales_train.loc[sales_train['location'] == '!Якутск', 'location'] = 'Якутск'
sales_train.loc[sales_train['location'] == 'Цифровой', 'location'] = 'Интернет-магазин'



def shop_type(row):
    inter = re.search('\s[\w\-]+\s\"[\s\w\-]*"', row)
    
    if inter is None:
        return 'Прочие'
    
    result = inter.group().split()[0]
    
    return result

sales_train['shop_type'] = sales_train['shop_location'].apply(shop_type)
sales_train.loc[sales_train['shop_type'] == 'Сити-Парк', 'shop_type'] = 'ТРЦ'

sales_train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,shop_location,item_category,item_name,day,month,dayofweek,year,date_month,category,subcategory,location,shop_type
0,2013-01-01,0,18,5823,2500.0,1.0,"Красноярск ТЦ ""Июнь""",Карты оплаты - PSN,Playstation Store пополнение бумажника: Карта ...,1,1,Tuesday,2013,2013-01,Карты оплаты,PSN,Красноярск,ТЦ
1,2013-01-01,0,27,5573,849.0,1.0,"Москва ТЦ ""МЕГА Белая Дача II""",Аксессуары - PS3,PS Move Motion Controller Charger (Зарядная ст...,1,1,Tuesday,2013,2013-01,Аксессуары,PS3,Москва,ТЦ
2,2013-01-01,0,7,1006,399.0,1.0,"Воронеж ТРЦ ""Максимир""",Подарки - Развитие,3D Crystal Puzzle Дельфин XL,1,1,Tuesday,2013,2013-01,Подарки,Развитие,Воронеж,ТРЦ
3,2013-01-01,0,19,17707,899.0,1.0,"Курск ТЦ ""Пушкинский""",Игры - PS3,Праздник спорта (Essentials) (только для PS Mo...,1,1,Tuesday,2013,2013-01,Игры,PS3,Курск,ТЦ
4,2013-01-01,0,14,19548,149.0,1.0,"Казань ТЦ ""ПаркХаус"" II",Кино - DVD,ТАКИЕ РАЗНЫЕ БЛИЗНЕЦЫ (регион),1,1,Tuesday,2013,2013-01,Кино,DVD,Казань,ТЦ


## 2. Plots

In [ ]:
plt.figure(figsize=(6, 6))

sns.scatterplot(x=sales_train['item_price'], y=sales_train['item_cnt_day'])

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

sns.boxplot(sales_train['item_price'], ax=ax1, whis=99)
sns.boxplot(sales_train['item_cnt_day'], ax=ax2, whis=99)

In [ ]:
plt.figure(figsize=(12, 7))

x = sales_train.groupby('dayofweek')['item_cnt_day'].mean().sort_values().index
y = sales_train.groupby('dayofweek')['item_cnt_day'].mean().sort_values().values - 1.15

sns.barplot(x=x, y=y)

plt.title('Distribution for "item_cnt_day" by date.')

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(15, 15))

x = sales_train.groupby('month')['item_cnt_day'].mean().index
y = sales_train.groupby('month')['item_cnt_day'].mean().values - 1

sns.barplot(x=x, y=y, ax=ax1)

x = sales_train.groupby('month')['item_price'].mean().index
y = sales_train.groupby('month')['item_price'].mean().values - 770

sns.barplot(x=x, y=y, ax=ax2)

ax1.set_title('Distribution for "item_cnt_day" by date.')

In [ ]:
plt.figure(figsize=(12, 7))

x = sales_train.groupby('year')['item_cnt_day'].mean().index
y = sales_train.groupby('year')['item_cnt_day'].mean().values - 1.22

sns.barplot(x=x, y=y)

plt.title('Distribution for "item_cnt_day" by date.')

In [ ]:
plt.figure(figsize=(12, 7))

x = sales_train.groupby('shop_type')['item_cnt_day'].mean().sort_values().index
y = sales_train.groupby('shop_type')['item_cnt_day'].mean().sort_values().values - 1.1

sns.barplot(x=x, y=y)

plt.title('Distribution for "item_cnt_day" by date.')
_ = plt.xticks(rotation=45)

In [ ]:
plt.figure(figsize=(12, 7))

x = sales_train.groupby('category')['item_cnt_day'].mean().sort_values().index
y = sales_train.groupby('category')['item_cnt_day'].mean().sort_values().values

sns.barplot(x=x, y=y)

plt.title('Distribution for "item_cnt_day" by date.')
_ = plt.xticks(rotation=45)

In [ ]:
plt.figure(figsize=(12, 7))

x = sales_train.groupby('location')['item_cnt_day'].mean().sort_values().index
y = sales_train.groupby('location')['item_cnt_day'].mean().sort_values().values

sns.barplot(x=x, y=y)

plt.title('Distribution for "item_cnt_day" by date.')
_ = plt.xticks(rotation=45)

## 3. ERA

In [7]:
sales_train.drop(['shop_location', 'item_category', 'item_name'], axis=1, inplace=True)
sales_train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,day,month,dayofweek,year,date_month,category,subcategory,location,shop_type
0,2013-01-01,0,18,5823,2500.0,1.0,1,1,Tuesday,2013,2013-01,Карты оплаты,PSN,Красноярск,ТЦ
1,2013-01-01,0,27,5573,849.0,1.0,1,1,Tuesday,2013,2013-01,Аксессуары,PS3,Москва,ТЦ
2,2013-01-01,0,7,1006,399.0,1.0,1,1,Tuesday,2013,2013-01,Подарки,Развитие,Воронеж,ТРЦ
3,2013-01-01,0,19,17707,899.0,1.0,1,1,Tuesday,2013,2013-01,Игры,PS3,Курск,ТЦ
4,2013-01-01,0,14,19548,149.0,1.0,1,1,Tuesday,2013,2013-01,Кино,DVD,Казань,ТЦ


### 3.1 Group by Month

In [8]:
_ = sales_train.pivot_table(index=['shop_id', 'item_id', 'date_month'], values=['item_price', 'item_cnt_day'], aggfunc={'item_price': np.mean, 'item_cnt_day': sum})
_.columns = ['item_cnt_month', 'item_price_month']
_ = _.reset_index()
_

,shop_id,item_id,date_month,item_cnt_month,item_price_month
0,0,30,2013-02,31.0,265.0
1,0,31,2013-02,11.0,434.0
2,0,32,2013-01,6.0,221.0
3,0,32,2013-02,10.0,221.0
4,0,33,2013-01,3.0,347.0
...,...,...,...,...,...
1609119,59,22164,2015-04,2.0,699.0
1609120,59,22164,2015-07,1.0,699.0
1609121,59,22167,2013-10,1.0,299.0
1609122,59,22167,2013-12,2.0,299.0


In [9]:
sales_train = sales_train.merge(_, how='inner', on=['shop_id', 'item_id', 'date_month'])
sales_train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,day,month,dayofweek,year,date_month,category,subcategory,location,shop_type,item_cnt_month,item_price_month
0,2013-01-01,0,18,5823,2500.0,1.0,1,1,Tuesday,2013,2013-01,Карты оплаты,PSN,Красноярск,ТЦ,3.0,2500.000000
1,2013-01-03,0,18,5823,2500.0,1.0,3,1,Thursday,2013,2013-01,Карты оплаты,PSN,Красноярск,ТЦ,3.0,2500.000000
2,2013-01-28,0,18,5823,2500.0,1.0,28,1,Monday,2013,2013-01,Карты оплаты,PSN,Красноярск,ТЦ,3.0,2500.000000
3,2013-01-01,0,27,5573,849.0,1.0,1,1,Tuesday,2013,2013-01,Аксессуары,PS3,Москва,ТЦ,8.0,913.285714
4,2013-01-02,0,27,5573,849.0,1.0,2,1,Wednesday,2013,2013-01,Аксессуары,PS3,Москва,ТЦ,8.0,913.285714


In [10]:
sales_train['discount'] = sales_train['item_price_month'] > sales_train['item_price']
sales_train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,day,month,dayofweek,year,date_month,category,subcategory,location,shop_type,item_cnt_month,item_price_month,discount
0,2013-01-01,0,18,5823,2500.0,1.0,1,1,Tuesday,2013,2013-01,Карты оплаты,PSN,Красноярск,ТЦ,3.0,2500.000000,False
1,2013-01-03,0,18,5823,2500.0,1.0,3,1,Thursday,2013,2013-01,Карты оплаты,PSN,Красноярск,ТЦ,3.0,2500.000000,False
2,2013-01-28,0,18,5823,2500.0,1.0,28,1,Monday,2013,2013-01,Карты оплаты,PSN,Красноярск,ТЦ,3.0,2500.000000,False
3,2013-01-01,0,27,5573,849.0,1.0,1,1,Tuesday,2013,2013-01,Аксессуары,PS3,Москва,ТЦ,8.0,913.285714,True
4,2013-01-02,0,27,5573,849.0,1.0,2,1,Wednesday,2013,2013-01,Аксессуары,PS3,Москва,ТЦ,8.0,913.285714,True


### 3.2 Mean Encoding

In [14]:
def mean_encoding_regression(sales_train, columns: [str], value: str):
    for column in columns:
        _ = sales_train.pivot_table(index=column, values=value).reset_index()
        sales_train = sales_train.merge(_, how='inner', on=[column], suffixes=['', f'_mean_{column}'])
    
    return sales_train

In [15]:
sales_train = mean_encoding_regression(sales_train, ['location', 'category', 'subcategory', 'dayofweek', 'shop_type'], 'item_cnt_day')
sales_train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,day,month,dayofweek,year,...,location,shop_type,item_cnt_month,item_price_month,discount,item_cnt_day_mean_location,item_cnt_day_mean_category,item_cnt_day_mean_subcategory,item_cnt_day_mean_dayofweek,item_cnt_day_mean_shop_type
0,2013-01-01,0,18,5823,2500.0,1.0,1,1,Tuesday,2013,...,Красноярск,ТЦ,3.0,2500.000000,False,1.19884,1.406449,1.348701,1.315853,1.223092
1,2013-01-01,0,18,5822,999.0,2.0,1,1,Tuesday,2013,...,Красноярск,ТЦ,40.0,994.583333,False,1.19884,1.406449,1.348701,1.315853,1.223092
2,2013-01-08,0,18,5822,999.0,4.0,8,1,Tuesday,2013,...,Красноярск,ТЦ,40.0,994.583333,False,1.19884,1.406449,1.348701,1.315853,1.223092
3,2013-01-22,0,18,5822,999.0,5.0,22,1,Tuesday,2013,...,Красноярск,ТЦ,40.0,994.583333,False,1.19884,1.406449,1.348701,1.315853,1.223092
4,2013-02-19,1,18,5822,1049.0,1.0,19,2,Tuesday,2013,...,Красноярск,ТЦ,15.0,1048.966667,False,1.19884,1.406449,1.348701,1.315853,1.223092


# CHECK IN

In [16]:
copy_saved_sales_train = sales_train.copy()

In [51]:
sales_train = copy_saved_sales_train

### 3.3 Add Past Month shop/item value and Past Year shop/item value

In [52]:
sales_train['date'].describe()

<ipython-input-52-f88335fc6981>:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  sales_train['date'].describe()


count                 2935843
unique                   1034
top       2013-12-28 00:00:00
freq                     9434
first     2013-01-01 00:00:00
last      2015-10-31 00:00:00
Name: date, dtype: object

In [ ]:
_ = sales_train.pivot_table(index=['shop_id', 'item_id', 'date_month'], values='item_cnt_day').reset_index()
november_2013 = _[_['date_month'] == '2013-11']
november_2014 = _[_['date_month'] == '2014-11']
october = _[_['date_month'] == '2015-10']
october.drop('date_month', axis=1, inplace=True)
sales_train = sales_train.merge(october, how='left', on=['shop_id', 'item_id'], suffixes=['', '_october'])
# sales_train = sales_train.merge(november_2013, on=['shop_id', 'item_id', 'date_month'], suffixes=['', '_november_2013'])
# sales_train = sales_train.merge(november_2014, on=['shop_id', 'item_id', 'date_month'], suffixes=['', '_november_2014'])
sales_train.head()

/home/magleb/.local/lib/python3.8/site-packages/pandas/core/frame.py:4157: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [1]:
october

NameError: name 'october' is not defined

KeyError: "['date_month'] not found in axis"

In [44]:
sales_train.isnull().sum()

date                                   0
date_block_num                         0
shop_id                                0
item_id                                0
item_price                             0
item_cnt_day                           0
day                                    0
month                                  0
dayofweek                              0
year                                   0
date_month                             0
category                               0
subcategory                            0
location                               0
shop_type                              0
item_cnt_month                         0
item_price_month                       0
discount                               0
item_cnt_day_mean_location             0
item_cnt_day_mean_category             0
item_cnt_day_mean_subcategory          0
item_cnt_day_mean_dayofweek            0
item_cnt_day_mean_shop_type            0
date_month_october               2363950
item_cnt_day_oct

In [36]:
sales_train['date'].describe()

<ipython-input-36-f88335fc6981>:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  sales_train['date'].describe()


count                 2935843
unique                   1034
top       2013-12-28 00:00:00
freq                     9434
first     2013-01-01 00:00:00
last      2015-10-31 00:00:00
Name: date, dtype: object

### Work with timeseries

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose

In [ ]:
_ = sales_train.pivot_table(index='date_month', values='item_cnt_day').reset_index()
_.plot(x='date_month', y='item_cnt_day')

In [ ]:
_['item_cnt_day'].shift().rolling(5).mean()

In [ ]:
sales_train['rolling_mean'] = sales_train['item_cnt_day'].shift().rolling(5).mean()

for i in range(1, 7):
    name_of_column = 'lag_' + str(i) 
    new_data_for_train[name_of_column] = new_data_for_train['num_orders'].shift(i)

In [ ]:
decomposed = seasonal_decompose(_.set_index('date_month'), period=12)
trend = decomposed.trend
seasonal = decomposed.seasonal
resid = decomposed.resid

In [ ]:
trend.plot()

In [ ]:
seasonal.plot()

In [ ]:
resid.plot()

In [ ]:
_ = sales_train.pivot_table(index='dayofweek', values='item_cnt_day')
_ = _.reindex(['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'])
_.reset_index().plot(x='dayofweek', y='item_cnt_day')

In [ ]:
sales_train.pivot_table(index='month', values='item_cnt_day').reset_index().plot(x='month', y='item_cnt_day')

In [ ]:
def save_submission(name, prediction):
    submission = pd.DataFrame(columns=['ID', 'item_cnt_month'])
    submission['item_cnt_month'] = prediction
    submission['ID'] = submission.index
    submission.to_csv(name, index=False)
    
    print("File was submitted to path: {}".format(name))
    

In [ ]:
sample_submission['item_cnt_month'] = 1
save_submission('sample_sub_1.csv', sample_submission['item_cnt_month'])